# Markdown → Google Doc (Google Docs API)

This notebook creates a Google Doc from the provided markdown notes and applies formatting (headings, bullets, checklist items, styled @mentions, footer styling).


In [ ]:
# Install dependencies (Colab usually has most of these, but this makes it reproducible)
!pip -q install google-api-python-client google-auth


In [ ]:
# Colab authentication
from google.colab import auth
auth.authenticate_user()

import google.auth
from googleapiclient.discovery import build

# Request the Docs scope explicitly
creds, _ = google.auth.default(scopes=["https://www.googleapis.com/auth/documents"])
docs = build("docs", "v1", credentials=creds)

print("Authenticated.")


In [ ]:
# Load the markdown (option A: keep as a string)
md_text = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""
print("Loaded markdown.")


In [ ]:
\
import re
from googleapiclient.errors import HttpError

MENTION_RE = re.compile(r"@\w+")

def parse_markdown(md_text: str):
    blocks = []
    for line in md_text.splitlines():
        raw = line.rstrip("\n")

        if not raw.strip():
            blocks.append({"kind":"blank", "text":"", "level":0})
            continue

        if raw.startswith("# "):
            blocks.append({"kind":"h1", "text":raw[2:].strip(), "level":0})
            continue
        if raw.startswith("## "):
            blocks.append({"kind":"h2", "text":raw[3:].strip(), "level":0})
            continue
        if raw.startswith("### "):
            blocks.append({"kind":"h3", "text":raw[4:].strip(), "level":0})
            continue

        if re.match(r"^\s*-\s\[( |x|X)\]\s+", raw):
            text = re.sub(r"^\s*-\s\[( |x|X)\]\s+", "", raw).strip()
            level = (len(raw) - len(raw.lstrip(" "))) // 2
            blocks.append({"kind":"checkbox", "text":text, "level":level})
            continue

        if re.match(r"^\s*[-*]\s+", raw):
            text = re.sub(r"^\s*[-*]\s+", "", raw).strip()
            level = (len(raw) - len(raw.lstrip(" "))) // 2
            blocks.append({"kind":"bullet", "text":text, "level":level})
            continue

        if raw.strip() == "---":
            blocks.append({"kind":"blank", "text":"", "level":0})
            continue

        if raw.startswith("Meeting recorded by:") or raw.startswith("Duration:"):
            blocks.append({"kind":"footer", "text":raw.strip(), "level":0})
            continue

        blocks.append({"kind":"p", "text":raw.strip(), "level":0})

    return blocks

def paragraph_style_request(start, end, named_style_type=None, indent_pt=None):
    style = {}
    fields = []
    if named_style_type:
        style["namedStyleType"] = named_style_type
        fields.append("namedStyleType")
    if indent_pt is not None:
        style["indentStart"] = {"magnitude": indent_pt, "unit":"PT"}
        fields.append("indentStart")
    return {
        "updateParagraphStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "paragraphStyle": style,
            "fields": ",".join(fields),
        }
    }

def text_style_request(start, end, bold=None, italic=None, font_size_pt=None, rgb=None):
    style = {}
    fields = []
    if bold is not None:
        style["bold"] = bold; fields.append("bold")
    if italic is not None:
        style["italic"] = italic; fields.append("italic")
    if font_size_pt is not None:
        style["fontSize"] = {"magnitude": font_size_pt, "unit":"PT"}
        fields.append("fontSize")
    if rgb is not None:
        style["foregroundColor"] = {"color":{"rgbColor": rgb}}
        fields.append("foregroundColor")
    return {
        "updateTextStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "textStyle": style,
            "fields": ",".join(fields),
        }
    }

def create_formatted_doc(md_text: str, title: str = "Product Team Sync (Generated)"):
    doc = docs.documents().create(body={"title": title}).execute()
    doc_id = doc["documentId"]
    url = f"https://docs.google.com/document/d/{doc_id}/edit"

    blocks = parse_markdown(md_text)

    cursor = 1
    parts = []
    spans = []
    for b in blocks:
        start = cursor
        line = b["text"]
        parts.append(line + "\n")
        cursor += len(line) + 1
        end = cursor
        spans.append({**b, "start":start, "end":end})

    full_text = "".join(parts)

    requests = [{"insertText": {"location":{"index": 1}, "text": full_text}}]

    # headings + footer
    for s in spans:
        if s["kind"] == "h1":
            requests.append(paragraph_style_request(s["start"], s["end"], named_style_type="HEADING_1"))
        elif s["kind"] == "h2":
            requests.append(paragraph_style_request(s["start"], s["end"], named_style_type="HEADING_2"))
        elif s["kind"] == "h3":
            requests.append(paragraph_style_request(s["start"], s["end"], named_style_type="HEADING_3"))
        elif s["kind"] == "footer":
            requests.append(text_style_request(
                s["start"], s["end"],
                italic=True, font_size_pt=10,
                rgb={"red":0.4, "green":0.4, "blue":0.4},
            ))

    # bullets + checkboxes
    for s in spans:
        if s["kind"] == "bullet":
            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": s["start"], "endIndex": s["end"]},
                    "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                }
            })
            if s["level"] > 0:
                requests.append(paragraph_style_request(s["start"], s["end"], indent_pt=18*s["level"]))

        elif s["kind"] == "checkbox":
            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": s["start"], "endIndex": s["end"]},
                    "bulletPreset": "BULLET_CHECKBOX"
                }
            })
            if s["level"] > 0:
                requests.append(paragraph_style_request(s["start"], s["end"], indent_pt=18*s["level"]))

    # @mentions
    for s in spans:
        for m in MENTION_RE.finditer(s.get("text","")):
            a = s["start"] + m.start()
            b = s["start"] + m.end()
            requests.append(text_style_request(a, b, bold=True))

    try:
        docs.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()
    except HttpError as e:
        raise RuntimeError(f"Google API error: {e}") from e

    return url

doc_url = create_formatted_doc(md_text)
print("Created document:", doc_url)
